In [1]:
using QuantEcon
using Optim
using BasisMatrices
using Plots

In [2]:
type MarketingBoardModel
    kappa::Float64　#費用定数
    sigma::Float64 #生産ショックにかかる定数
    delta::Float64 #割引定数
    gamma::Float64 #状態遷移関数の係数
    x_max::Float64 #最大保存量
    snodes::Array{Float64}
end

In [3]:
kappa = 0.05 
gamma = -0.5
mu = log([0.5; 0.5])
sigma = 0.2
delta = 0.95
x_max = 0.5

0.5

In [4]:
nshocks = [3; 3]
cov = (sigma^2)*eye(2)
epsilon, weight = qnwlogn(nshocks, mu, cov)

(
[0.353611 0.353611; 0.5 0.353611; … ; 0.5 0.706991; 0.706991 0.706991],

[0.0277778,0.111111,0.0277778,0.111111,0.444444,0.111111,0.0277778,0.111111,0.0277778])

In [5]:
n = [20; 20]
s_min = minimum(epsilon)

0.35361117610946247

In [6]:
s_max = maximum(epsilon) + x_max　#ここでsmaxのベクトルができる

1.2069912290402582

In [7]:
basis =  Basis(SplineParams(n[1]-2, s_min, s_max), SplineParams(n[2]-2, s_min, s_max))
snodes, (s1_vec, s2_vec) = nodes(basis)

(
[0.353611 0.353611; 0.370344 0.353611; … ; 1.19026 1.20699; 1.20699 1.20699],

([0.353611,0.370344,0.40381,0.454009,0.504208,0.554406,0.604605,0.654804,0.705003,0.755202,0.805401,0.855599,0.905798,0.955997,1.0062,1.05639,1.10659,1.15679,1.19026,1.20699],[0.353611,0.370344,0.40381,0.454009,0.504208,0.554406,0.604605,0.654804,0.705003,0.755202,0.805401,0.855599,0.905798,0.955997,1.0062,1.05639,1.10659,1.15679,1.19026,1.20699]))

In [8]:
Φ = BasisMatrix(basis, Expanded(), snodes, 0)
initial_c = Φ.vals[1] \ ones(size(Φ.vals[1])[1])

400-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮  
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [9]:
MBM = MarketingBoardModel(kappa, sigma, delta, gamma, x_max, snodes)

MarketingBoardModel(0.05,0.2,0.95,-0.5,0.5,[0.353611 0.353611; 0.370344 0.353611; … ; 1.19026 1.20699; 1.20699 1.20699])

In [10]:
function q(MBM::MarketingBoardModel, s::Array{Float64, 2}, x::Array{Float64, 2})
    return s[:,1]-x[:,1] + s[:,2]-x[:,2]
end

q (generic function with 1 method)

In [11]:
function p(MBM::MarketingBoardModel, s::Array{Float64, 2}, x::Array{Float64, 2})
    return q(MBM, s, x).^MBM.gamma
end

p (generic function with 1 method)

In [42]:
function f(MBM::MarketingBoardModel, s::Array{Float64, 2}, x::Array{Float64, 2}, k::Int64)
    return p(MBM, s, x).*(s[:,k]-x[:,k]) - MBM.kappa*x[:,k]
end

f (generic function with 1 method)

In [62]:
function update_Bellman(MBM::MarketingBoardModel, C::Vector)
    kappa, sigma, delta, gamma, snodes = MBM.kappa, MBM.sigma, MBM.delta, MBM.gamma, MBM.snodes
    V_new = Array{Float64, 2}(size(snodes))
    x_opt = Array{Float64, 2}(size(snodes))
    for k in 1:2
        for i in 1:size(snodes, 1)
            objective(x::Array{Float64, 2}) = -(f(MBM, snodes, x, k) 
                + delta * dot(weight, [funeval( C, basis, [x[i,1]+epsilon[j,1] , x[i,2]+epsilon[j,2]] ) for j in 1:size(epsilon, 1)]))
            opt = optimize(objective, zeros(2) .+ 0.01, [s_min, 0], [s_max 0; 0 x_max] * ones(2), Fminbox{}())
            V_new[i, k] = opt.minimum
            x_opt[i, k] = opt.minimizer    
        end
            C_new[k] = Φ.vals[1] \ V_new
    end
    return C_new, x_opt
    
end

update_Bellman (generic function with 1 method)

In [23]:
a = zeros(400, 2)
dot(weight, [funeval( C, basis, [a[i,1]+epsilon[j,1] , a[i,2]+epsilon[j,2]] ) for j in 1:size(epsilon, 1) ] )

LoadError: BoundsError: attempt to access 400×2 Array{Float64,2} at index [0,1]

In [31]:
for i in 1:size(snodes, 1)
    return f(MBM, snodes, a, 1) + MBM.delta*dot(weight, [funeval( C, basis, [a[i,1]+epsilon[j,1] , a[i,2]+epsilon[j,2]] ) for j in 1:size(epsilon, 1) ])
end

400-element Array{Float64,1}:
 1.37048
 1.38526
 1.41399
 1.4552 
 1.49439
 1.53181
 1.56765
 1.60207
 1.63521
 1.66719
 1.69811
 1.72807
 1.75714
 ⋮      
 1.45986
 1.48913
 1.51775
 1.54575
 1.57317
 1.60002
 1.62635
 1.65218
 1.67752
 1.7024 
 1.71875
 1.72685

LoadError: UndefVarError: b not defined

In [16]:
C = initial_c
tol = 0.000000001
max_iter = 200
C_error = 1.0
i = 0
x_opt = Array{Float64, 2}(size(snodes))

400×2 Array{Float64,2}:
 0.0           4.94066e-324
 6.36599e-314  0.0         
 6.95314e-310  0.0         
 1.4822e-322   7.64001e-313
 0.0           6.95198e-310
 0.0           6.95198e-310
 0.0           6.95198e-310
 6.36599e-314  9.33678e-313
 6.95314e-310  6.95198e-310
 1.58101e-322  6.95198e-310
 0.0           6.95198e-310
 2.37152e-322  6.95198e-310
 0.0           6.95198e-310
 ⋮                         
 6.95198e-310  2.44063e-312
 2.22603e-314  6.95198e-310
 6.95198e-310  0.0         
 6.95198e-310  6.95198e-310
 5.45353e-312  6.95198e-310
 0.0           6.95198e-310
 0.0           6.95198e-310
 6.95198e-310  4.94066e-324
 6.95198e-310  6.95198e-310
 2.22597e-314  2.122e-314  
 6.95198e-310  6.95198e-310
 6.95198e-310  0.0         

In [63]:
while C_error > tol && i <= max_iter
    C_computed, x_opt_computed = update_Bellman(MBM, C)
    C_error = maximum(abs, C_computed - C)
    copy!(C, C_computed)
    copy!(x_opt, x_opt_computed)
    i += 1
end

i, C_error

LoadError: MethodError: no method matching optimize(::#objective#74{MarketingBoardModel,Array{Float64,1},Float64,Array{Float64,2}}, ::Array{Float64,1}, ::Array{Float64,1}, ::Array{Float64,1}, ::Optim.Fminbox)[0m
Closest candidates are:
  optimize{T<:AbstractFloat}([1m[31m::Optim.OnceDifferentiable[0m, ::Array{T<:AbstractFloat,N}, ::Array{T<:AbstractFloat,N}, ::Array{T<:AbstractFloat,N}, ::Optim.Fminbox; x_tol, f_tol, g_tol, allow_f_increases, iterations, store_trace, show_trace, extended_trace, callback, show_every, linesearch!, linesearch, eta, mu0, mufactor, precondprep!, precondprep, optimizer, optimizer_o, nargs...) at /Users/sasakiakihiro/.julia/v0.5/Optim/src/fminbox.jl:136
  optimize{F<:Function,G<:Function,H<:Function}(::F<:Function, [1m[31m::G<:Function[0m, [1m[31m::H<:Function[0m, ::Array{T,N}, ::Optim.Optimizer) at /Users/sasakiakihiro/.julia/v0.5/Optim/src/optimize.jl:75
  optimize{F<:Function,G<:Function,H<:Function}(::F<:Function, [1m[31m::G<:Function[0m, [1

In [18]:
MBM.delta * dot(weight, [funeval( C, basis, [x[i,1]+epsilon[j,1] , x[i,2]+epsilon[j,2]] ) for j in 1:size(epsilon, 1) ] )

LoadError: UndefVarError: x not defined